Python Program to learn how to process spreadsheets

In [1]:
import openpyxl as xl 
from openpyxl.chart import BarChart , Reference

In [32]:
wb = xl.load_workbook('transactions.xlsx')
sheet = wb['Sheet1'] # access any specific sheet 
# cell = sheet['A1'] # to access a particular cell  give the coordinate of a cell.
cell = sheet.cell(1,1) # position of the row & column
print(cell.value)

transaction_id


to iterate over all the rows of the sheet , we need to know how many rows we have in this spreadsheet

In [33]:
print(sheet.max_row)
print(sheet.max_column)

4
3


In [34]:
for row in range(2,sheet.max_row + 1):
    cell = sheet.cell(row,3) # access each cell of 3rd column
    print(cell.value)

5.95
6.95
7.95


we want to decrease 10% of the prices .

In [35]:
for row in range(2,sheet.max_row + 1):
    cell = sheet.cell(row,3) # access each cell of 3rd column
    corrected_price = cell.value * 0.9
    print(corrected_price)
    corrected_price_cell = sheet.cell(row,4) # this returns a cell object
    corrected_price_cell.value = corrected_price

wb.save('transactions2.xlsx')
  

5.355
6.255
7.155


Create a Bar chart with the values of 4th column 

In [36]:
value = Reference(sheet,min_row=2,max_row= sheet.max_row ,min_col=4,max_col=4) # returns all the values of 4th column
chart = BarChart()
chart.add_data(value)
sheet.add_chart(chart,'F2')
wb.save('transactions2.xlsx')


if we are going to use this to automate the process of updating thousands of spreadsheets, this program wouldn't work, because it's only relying on one file. So we have to reorganize this code, and move it inside of a function. This function should take the name of the file as input .

In [ ]:
def process_workbook(filename , sheet_name ,col ,col_name, change_percentage ):
    wb = xl.load_workbook(filename)
    sheet = wb[sheet_name] # access any specific sheet 
    
    # Insert new column right after the original price column
    sheet.insert_cols(col + 1)
    sheet.cell(row=1, column=col + 1).value = col_name
    # sheet.cell(row=1, column=col + 1 ,value = col_name)

    # Fill corrected values
    for row in range(2,sheet.max_row + 1):
        cell = sheet.cell(row,col) # access each cell of specified column
        corrected_value = cell.value * change_percentage if isinstance(cell.value, (int, float)) else None
        if corrected_value is not None:
            sheet.cell(row=row, column=col + 1).value = corrected_value
    
    # Create bar chart referencing the corrected column
    value = Reference(sheet,min_row=2,max_row= sheet.max_row ,min_col= col + 1,max_col= col + 1) # returns all the values of specified column
    chart = BarChart()
    chart.add_data(value ,titles_from_data=True)
    sheet.add_chart(chart,'F2')
    
    # Save to a new file
    wb.save(filename.replace(".xlsx", "_processed.xlsx"))
    

In [13]:
process_workbook(
    filename="transactions.xlsx",
    sheet_name="Sheet1",
    col=3,
    col_name="Corrected Price",
    change_percentage=0.8
)

To process multiple spreadsheets

In [28]:
import openpyxl as xl
from openpyxl.chart import BarChart, Reference
from openpyxl.styles import Font, PatternFill, Alignment  # for styles and formatting
import os
from statistics import mean  # to calculate average easily

def process_workbook(filename , sheet_name , col , col_name , change_percentage):
    # Load the workbook and access the correct sheet
    wb = xl.load_workbook(filename)
    sheet = wb[sheet_name]   # access any specific sheet
    
    # Insert a new column right after the original column (col+1 is "Corrected Price")
    sheet.insert_cols(col + 1)
    sheet.cell(row=1, column=col + 1).value = col_name
        # sheet.cell(row=1, column=col + 1 ,value = col_name)


    # Style the new header cell (bold, colored, centered)
    header_cell = sheet.cell(row=1, column=col + 1)
    header_cell.font = Font(bold=True, color="FFFFFF")        # White bold text
    header_cell.fill = PatternFill(start_color="4F81BD",      # Blue cell background
                                   end_color="4F81BD", fill_type="solid")
    header_cell.alignment = Alignment(horizontal="center")    # Center align text

    # Create a list to keep "corrected price" values for summaries later
    corrected_prices = []

    # ====  Fill corrected values ====
    for row in range(2, sheet.max_row + 1):
        cell = sheet.cell(row, col)   # original price cell and access each cell of specified column
        value = cell.value # related to Skip blanks

        # Skip blanks
        if value is None:
            print(f"⚠️ {filename} - Row {row}: Missing value, skipped.")
            continue
        # Skip wrong data types (must be number, not text like "N/A")
        if not isinstance(value, (int, float)):
            print(f"⚠️ {filename} - Row {row}: Invalid data '{value}', skipped.")
            continue

        # Apply discount or price change
        corrected_value = value * change_percentage
        sheet.cell(row=row, column=col + 1).value = corrected_value

        # Apply currency style (shows as ৳ 1,234.50)
        sheet.cell(row=row, column=col + 1).number_format = '"৳"#,##0.00'

        # Save for summary totals later
        corrected_prices.append(corrected_value)

    # ==== Chart comparing Original vs Corrected ====
    # Reference both columns (original and corrected)
    values = Reference(sheet, min_row=1, max_row=sheet.max_row, min_col=col, max_col=col+1) # returns all the values of specified column
    
    # Create a bar chart with two sets of data
    chart = BarChart()
    chart.add_data(values, titles_from_data=True)
    chart.title = "Original vs Corrected Prices"  # Chart title
    chart.y_axis.title = "Price"                 # Y-axis label
    sheet.add_chart(chart, 'G2')                 # Add chart starting at cell G2

    # ==== Summary Report ====
    # If we got valid corrected prices, create summary results
    if corrected_prices:
        total_corrected = sum(corrected_prices)      # get total
        average_corrected = mean(corrected_prices)   # get average

        # Place the summary 2 rows below the data
        summary_row = sheet.max_row + 2

        # Write "Total Corrected Price" in one column
        sheet.cell(row=summary_row, column=col).value = "Total Corrected Price"
        sheet.cell(row=summary_row, column=col).font = Font(bold=True)
        sheet.cell(row=summary_row, column=col + 1).value = total_corrected
        sheet.cell(row=summary_row, column=col + 1).number_format = '"৳"#,##0.00'

        # Write "Average Corrected Price" just below total
        sheet.cell(row=summary_row + 1, column=col).value = "Average Corrected Price"
        sheet.cell(row=summary_row + 1, column=col).font = Font(bold=True)
        sheet.cell(row=summary_row + 1, column=col + 1).value = average_corrected
        sheet.cell(row=summary_row + 1, column=col + 1).number_format = '"৳"#,##0.00'

        # Style the summary area with a light green background
        for r in (summary_row, summary_row + 1):
            label_cell = sheet.cell(row=r, column=col)
            label_cell.fill = PatternFill(start_color="C6EFCE",
                                          end_color="C6EFCE", fill_type="solid")
            
    # Save output with "_processed.xlsx" to avoid overwriting original file
    output_file = filename.replace(".xlsx", "_processed.xlsx")
    wb.save(output_file)
    print(f"💾 Saved {output_file}")

In [29]:
# Batch processing ====
# Loop through every Excel file in "data" folder
folder = r"C:\Users\mehed\Documents\Automation with Python"
for file in os.listdir(folder):
    if file.endswith(".xlsx") and not file.endswith("_processed.xlsx"):  
        filepath = os.path.join(folder, file)
        process_workbook(
            filename=filepath,
            sheet_name="Sheet1",
            col=3,  # original price column (3rd column in this dataset)
            col_name="Corrected Price",  # new column name
            change_percentage=0.8        # apply 20% discount
        )

💾 Saved C:\Users\mehed\Documents\Automation with Python\transactions_processed.xlsx
